In [1]:
from __future__ import print_function

Load in the Python modules necessary to load the data (gzip and yaml), process the data (numpy and pandas), and plot the data (toyplot).

In [2]:
import gzip
import yaml
import numpy
import pandas
import toyplot.pdf

print("yaml version:    ", yaml.__version__)
print("numpy version:   ", numpy.__version__)
print("pandas version:  ", pandas.__version__)
print("toyplot version: ", toyplot.__version__)

yaml version:     3.12
numpy version:    1.14.0
pandas version:   0.22.0
toyplot version:  0.16.0


## Ingest Data

Load the data, which is output into a YAML files.

In [3]:
filename = 'miniGraphics-skybridge-vn-scaling.yaml.gz'
yaml_data = yaml.load(gzip.open(filename))
data_vn = pandas.DataFrame(yaml_data)

filename = 'miniGraphics-skybridge-smp-scaling.yaml.gz'
yaml_data = yaml.load(gzip.open(filename))
data_smp = pandas.DataFrame(yaml_data)

The YAML data is hierarchical. The basic yaml reader to DataFrame just embeds dictionaries and lists in DataFrame columns. Fix that by expanding the data of these columns into new columns.

In [4]:
def expand_single_column(original_data, column_to_expand):

    expanded_data = pandas.DataFrame()
    for index in original_data.index:
        sub_table = pandas.DataFrame(original_data[column_to_expand][index])
        for column in original_data.columns:
            if column != column_to_expand:
                sub_table[column] = numpy.full(sub_table.index.shape,
                                               original_data[column][index],
                                               dtype=original_data[column].dtype)
        expanded_data = expanded_data.append(sub_table, ignore_index=True)
    return expanded_data

def flatten_table(original_data):
    flat_data = original_data
    for column_name in original_data.columns:
        if isinstance(flat_data[column_name][0], list):
            flat_data = expand_single_column(flat_data, column_name)
    return flat_data

In [5]:
data_vn = flatten_table(data_vn)
data_smp = flatten_table(data_smp)

Add a column that gives a human-readable name to each image resolution.

In [6]:
image_height_names = {
    500: 'Desktop Window',
    1080: 'HDTV',
    4320: '8K UHD',
}

data_vn['image-size'] = data_vn['image-height'].map(image_height_names)
data_smp['image-size'] = data_smp['image-height'].map(image_height_names)

Rename the algorithms from the identifiers the program wrote out to the strings used in the paper. Note that there are some extras in the data that we are ignoring.

In [7]:
algorithm_names = {
    '2-3-SwapBase': '2-3 Swap',
    'BinarySwapFold': 'Naive',
    'BinarySwapTelescoping': 'Telescoping',
    'BinarySwapRemainder': 'Remainder',
    'IceTBase': 'IceT'
}

data_vn['composite-algorithm'] = data_vn['composite-algorithm'].map(algorithm_names)
data_smp['composite-algorithm'] = data_smp['composite-algorithm'].map(algorithm_names)

Print a summary of the table data. There are multiple ways that Jupyter and pandas will report a summary of a table, but I find this method the most effective. It prints out every column. Then for all columns with a "small" number of unique values, it gives those values. This latter information really helps identify the proper way to group values.

In [8]:
import IPython.display

data_description = '##### Virtual Node\n\n'

for column_name in data_vn.columns:
    data_description = data_description + '**' + column_name + '**: '
    unique_values = data_vn[column_name].unique()
    if (len(unique_values) < 10):
        for value in unique_values:
            data_description = data_description + str(value) + ' '
    else:
        data_description = (
            data_description +
            str(numpy.min(unique_values)) + ' &ndash; ' +
            str(numpy.max(unique_values)) + ' '
        )
    data_description = data_description + ' \n'
    
data_description = data_description + '\n##### SMP\n\n'

for column_name in data_smp.columns:
    data_description = data_description + '**' + column_name + '**: '
    unique_values = data_smp[column_name].unique()
    if (len(unique_values) < 10):
        for value in unique_values:
            data_description = data_description + str(value) + ' '
    else:
        data_description = (
            data_description +
            str(numpy.min(unique_values)) + ' &ndash; ' +
            str(numpy.max(unique_values)) + ' '
        )
    data_description = data_description + ' \n'
    
IPython.display.display(IPython.display.Markdown(data_description))

C:\Users\kmorel\AppData\Local\Continuum\Anaconda2\lib\site-packages\numpy\core\_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)
C:\Users\kmorel\AppData\Local\Continuum\Anaconda2\lib\site-packages\numpy\core\_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)


##### Virtual Node

**color-buffer-format**: byte  
**composite-algorithm**: Naive Remainder Telescoping nan 2-3 Swap IceT  
**composite-seconds**: 0.00227687 &ndash; 28.7058  
**compress-seconds**: 2.315e-06 &ndash; 8e-06  
**construct-tree-seconds**: nan &ndash; nan  
**depth-buffer-format**: float  
**gather-seconds**: 0.00095674 &ndash; 4.52609  
**geometry**: box  
**geometry-distribution**: duplicate  
**geometry-overlap**: -0.05  
**icet-copy-result-seconds**: 1.304e-06 &ndash; 5e-05  
**image-compression**: True False  
**image-height**: 500 1080 4320  
**image-width**: 500 1920 7680  
**k**: nan &ndash; 8,8,8,8  
**max-image-split**: nan 1000000.0  
**num-processes**: 64 &ndash; 8192  
**num-triangles**: 768 &ndash; 98304  
**paint-seconds**: 0.00020071 &ndash; 0.170633  
**painter**: simple  
**partial-composite-seconds**: 0.00037141 &ndash; 27.0723  
**phi-rotation**: -178.406 &ndash; 136.756  
**random-seed**: 17627  
**rendering-order-dependent**: False  
**start-time**: 2018-04-03T12:58:47.000000000 &ndash; 2018-04-18T10:04:49.000000000  
**theta-rotation**: -177.945 &ndash; 179.418  
**total-seconds**: 0.00280725 &ndash; 28.7062  
**trial-num**: 0 &ndash; 19  
**uncompress-seconds**: 1.337e-06 &ndash; 8e-06  
**zoom**: 1.5  
**image-size**: Desktop Window HDTV 8K UHD  

##### SMP

**color-buffer-format**: byte  
**composite-algorithm**: Naive Remainder Telescoping nan 2-3 Swap IceT  
**composite-seconds**: 0.00136872 &ndash; 0.903929  
**compress-seconds**: 8.137e-06 &ndash; 2e-05  
**construct-tree-seconds**: nan &ndash; nan  
**depth-buffer-format**: float  
**gather-seconds**: 0.00071163 &ndash; 0.51739  
**geometry**: box  
**geometry-distribution**: duplicate  
**geometry-overlap**: -0.05  
**icet-copy-result-seconds**: nan &ndash; nan  
**image-compression**: True False  
**image-height**: 500 1080 4320  
**image-width**: 500 1920 7680  
**k**: nan &ndash; 8,8,4  
**max-image-split**: nan 1000000.0  
**num-processes**: 33 &ndash; 512  
**num-triangles**: 396 &ndash; 6144  
**paint-seconds**: 0.000216328 &ndash; 0.126701  
**painter**: simple  
**partial-composite-seconds**: 0.000240892 &ndash; 0.61017  
**phi-rotation**: -178.406 &ndash; 136.756  
**random-seed**: 17627  
**rendering-order-dependent**: False  
**start-time**: 2018-04-05T16:27:11.000000000 &ndash; 2018-04-18T10:00:13.000000000  
**theta-rotation**: -177.945 &ndash; 179.418  
**total-seconds**: 0.00175639 &ndash; 1.00893  
**trial-num**: 0 &ndash; 19  
**uncompress-seconds**: 2.56e-06 &ndash; 7e-06  
**zoom**: 1.5  
**image-size**: Desktop Window HDTV 8K UHD  


## Plot Data

We are plotting the time it takes to do a "partial composite" (that is the time to blend all the pixels, but the pixels are left distributed across all the processes).

The first thing we want to do is to average the time it took over all trials. This is easily done with a pivot table. We also need the filter the data to those that have been run in both VN and SMP modes. These are those between 64 and 512 processes.

In [9]:
filtered_indices = (
    (data_vn['num-processes'] >= 64) &
    (data_vn['num-processes'] <= 512)
)

average_partial_composite_vn = data_vn[filtered_indices].pivot_table(
    values='partial-composite-seconds',
    index='num-processes',
    columns=[
        'image-size',
        'composite-algorithm',
    ],
    aggfunc='mean',
)

average_partial_composite_vn

image-size             8K UHD                                            \
composite-algorithm  2-3 Swap      IceT     Naive Remainder Telescoping   
num-processes                                                             
64                   0.385781  0.065904  0.378764  0.378830    0.379878   
96                   0.386223  0.050465  0.689278  0.369679    0.372970   
128                  0.376284  0.047220  0.366858  0.367040    0.364874   
144                  0.385339  0.036686  0.675675  0.365959    0.366791   
192                  0.377325  0.034736  0.671982  0.363636    0.365166   
256                  0.374036  0.033015  0.364563  0.364479    0.364077   
288                  0.381443  0.027747  0.675306  0.364104    0.365186   
384                  0.372013  0.024902  0.667428  0.360336    0.359958   
432                  0.379653  0.026139  0.669627  0.361070    0.363800   
512                  0.374163  0.024898  0.362974  0.361116    0.361790   

image-size          Desktop Window                                            \
composite-algorithm       2-3 Swap      IceT     Naive Remainder Telescoping   
num-processes                                                                  
64                        0.002578  0.001049  0.002421  0.002388    0.002378   
96                        0.002681  0.000905  0.004221  0.002380    0.002330   
128                       0.002604  0.000921  0.005175  0.002306    0.002314   
144                       0.002709  0.000783  0.004077  0.002313    0.002327   
192                       0.002777  0.000807  0.004090  0.002380    0.002322   
256                       0.002898  0.000787  0.002310  0.002299    0.002304   
288                       0.003062  0.000904  0.004009  0.002395    0.002362   
384                       0.003080  0.000727  0.004071  0.002317    0.002307   
432                       0.003319  0.000750  0.004149  0.002303    0.002315   
512                       0.003412  0.000764  0.002316  0.002321    0.002323   

image-size               HDTV                                            
composite-algorithm  2-3 Swap      IceT     Naive Remainder Telescoping  
num-processes                                                            
64                   0.021511  0.004144  0.020699  0.020650    0.020404  
96                   0.020162  0.003233  0.038309  0.020051    0.020226  
128                  0.020711  0.003177  0.019816  0.019754    0.019822  
144                  0.020820  0.002511  0.037759  0.019917    0.019812  
192                  0.020209  0.002371  0.037716  0.019785    0.019953  
256                  0.020870  0.002478  0.019719  0.019568    0.019753  
288                  0.020717  0.001973  0.037366  0.019705    0.019729  
384                  0.020187  0.001910  0.037300  0.019585    0.019639  
432                  0.021254  0.002065  0.037033  0.019693    0.019699  
512                  0.021579  0.002171  0.019696  0.019818    0.019739

In [10]:
filtered_indices = (
    (data_smp['num-processes'] >= 64) &
    (data_smp['num-processes'] <= 512)
)

average_partial_composite_smp = data_smp[filtered_indices].pivot_table(
    values='partial-composite-seconds',
    index='num-processes',
    columns=[
        'image-size',
        'composite-algorithm',
    ],
    aggfunc='mean',
)

average_partial_composite_smp

image-size             8K UHD                                            \
composite-algorithm  2-3 Swap      IceT     Naive Remainder Telescoping   
num-processes                                                             
64                   0.307841  0.037455  0.290259  0.288952    0.291212   
66                   0.311272  0.036759  0.535120  0.284088    0.281505   
72                   0.320871  0.034298  0.559619  0.282821    0.283425   
81                   0.314611  0.036029  0.561087  0.287900    0.284968   
96                   0.302945  0.028671  0.562406  0.281927    0.282976   
99                   0.313586  0.033798  0.556161  0.285803    0.283042   
108                  0.312559  0.027405  0.557984  0.278308    0.283300   
128                  0.292290  0.023212  0.275243  0.275089    0.275514   
132                  0.300673  0.023672  0.532675  0.272355    0.275149   
144                  0.316673  0.025793  0.550598  0.277241    0.278625   
162                  0.308419  0.023456  0.551522  0.278514    0.278506   
192                  0.297511  0.019180  0.547107  0.275894    0.277341   
198                  0.299436  0.019353  0.537969  0.270510    0.271397   
216                  0.312212  0.019844  0.553346  0.279217    0.281119   
243                  0.306598  0.019368  0.547294  0.273991    0.274856   
256                  0.290987  0.015650  0.270414  0.272230    0.271704   
288                  0.315753  0.017902  0.542737  0.273757    0.273843   
324                  0.309596  0.016610  0.542555  0.271941    0.271926   
384                  0.291715  0.013780  0.540572  0.270963    0.271860   
432                  0.303457  0.012746  0.539391  0.269881    0.270753   
486                  0.308379  0.012633  0.542164  0.270526    0.272941   
512                  0.291229  0.011520  0.270641  0.272013    0.272155   

image-size          Desktop Window                                            \
composite-algorithm       2-3 Swap      IceT     Naive Remainder Telescoping   
num-processes                                                                  
64                        0.002017  0.000498  0.001866  0.001876    0.001889   
66                        0.001964  0.000494  0.003213  0.001737    0.001777   
72                        0.002058  0.000456  0.003543  0.001804    0.001813   
81                        0.002049  0.000499  0.003569  0.001795    0.001803   
96                        0.001976  0.000450  0.003524  0.001810    0.001803   
99                        0.002043  0.000540  0.003474  0.001782    0.001761   
108                       0.002119  0.000425  0.003480  0.001727    0.001744   
128                       0.001915  0.000416  0.001697  0.001728    0.001724   
132                       0.002045  0.000433  0.003179  0.001714    0.001723   
144                       0.002159  0.000397  0.003516  0.001742    0.001753   
162                       0.002124  0.000395  0.003392  0.001753    0.001780   
192                       0.002089  0.000378  0.003371  0.001750    0.001738   
198                       0.002120  0.000392  0.003291  0.001687    0.001702   
216                       0.002305  0.000380  0.003470  0.001763    0.001767   
243                            NaN  0.000359  0.003402  0.001749    0.001891   
256                       0.002120  0.000356  0.001702  0.001705    0.001751   
288                       0.002413  0.000367  0.003445  0.001727    0.001742   
324                       0.002397  0.000350  0.003425  0.001692    0.001751   
384                       0.002364  0.000430  0.003411  0.001732    0.001755   
432                       0.002631  0.000351  0.003406  0.001690    0.001752   
486                            NaN  0.000352  0.003466  0.001720    0.001751   
512                       0.002751  0.000435  0.001750  0.001834    0.001881   

image-size               HDTV                                            
composite-algorithm  2-3 Swap      IceT     Naive

In [11]:
numpy.max(numpy.max(average_partial_composite_vn['HDTV']))

0.03830913

Make a grouping structure of the data so we can pull out the actual data values for each trial.

In [12]:
image_size = 'HDTV'

canvas = toyplot.Canvas('9.25in', '3in',
                        #style={'background-color': 'yellow'},
                       )

x_vn = average_partial_composite_vn.index
x_smp = average_partial_composite_smp.index

maxy = max(
    numpy.max(numpy.max(average_partial_composite_vn[image_size])),
    numpy.max(numpy.max(average_partial_composite_smp[image_size])),
)

xlocator = toyplot.locator.Log(base=2, format='{:.0f}')

def doPlot(axes, algorithm):
    axes.x.ticks.locator = xlocator
    axes.y.domain.min = 0
    axes.y.domain.max = maxy
    
    y_vn = numpy.array(average_partial_composite_vn[image_size, algorithm])
    y_smp = numpy.array(average_partial_composite_smp[image_size, algorithm])

    axes.plot(x_vn, y_vn)
    axes.plot(x_smp, y_smp)

    axes.text(
        512, numpy.max(y_vn), 'Virtual Node: ' + algorithm,
        style={
            'text-anchor': 'end',
            '-toyplot-vertical-align': 'last-baseline',
        }
    )
    axes.text(
        512, numpy.min(y_smp), 'Pure Distributed: ' + algorithm,
        style={
            'text-anchor': 'end',
            '-toyplot-vertical-align': 'top',
        }
    )

axesNaive = canvas.cartesian(
    xlabel = 'Number of Processes',
    ylabel = 'Partial Composite Time (seconds)',
    xscale='log',
    bounds=('5%', '33%', '5%', '85%'),
)
doPlot(axesNaive, 'Naive')

axes23 = canvas.cartesian(
    xlabel = 'Number of Processes',
    #ylabel = 'Partial Composite Time (seconds)',
    xscale='log',
    bounds=('38%', '66%', '5%', '85%'),
)
doPlot(axes23, '2-3 Swap')

axesRemainder = canvas.cartesian(
    xlabel = 'Number of Processes',
    #ylabel = 'Partial Composite Time (seconds)',
    xscale='log',
    bounds=('70%', '99%', '5%', '85%'),
)
doPlot(axesRemainder, 'Remainder')

<path d="M 44.400000000000006 158.5131264 L 48.07938461242868 85.77866880000002 L 58.4833841195389 75.0779904 L 72.56676823907773 73.750608 L 92.88169205976945 74.44168320000003 L 96.56107667219807 78.10997760000001 L 106.96507617930834 74.64005759999999 L 127.28000000000002 159.53704320000003 L 130.9593846124287 87.58540799999999 L 141.3633841195389 75.2874336 L 155.44676823907767 73.17406079999998 L 175.76169205976947 75.8889024 L 179.441076672198 85.41584639999999 L 189.84507617930825 73.70714880000004 L 203.9284602988471 75.82936319999995 L 210.16000000000003 159.90055679999998 L 224.24338411953892 74.841264 L 238.3267682390777 73.9319904 L 258.64169205976947 74.48904000000003 L 272.72507617930825 74.64090240000004 L 286.80846029884714 75.07359359999998 L 293.04 158.5210464" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> Virtual Node: Naive Pure Distributed: Naive 64 128 256 512 Number of Processes 0.00 0.01 0.02 0.03 0.04 Partial Composite Time (seconds) <path d="M 337.44 149.36261759999996 L 341.1193846124287 150.48587519999998 L 351.5233841195389 146.7896448 L 365.60676823907767 146.9369088 L 385.92169205976944 150.35049599999996 L 389.601076672198 146.6138304 L 400.0050761793084 146.09996159999997 L 420.32000000000005 152.7081024 L 423.9993846124287 150.70590719999998 L 434.40338411953894 146.11893120000002 L 448.4867682390777 147.07244160000002 L 468.8016920597695 150.9171264 L 472.481076672198 149.15836799999997 L 482.88507617930827 143.22830399999998 L 496.96846029884716 149.5694016 L 503.20000000000005 151.58858879999997 L 517.2833841195389 145.7156448 L 531.3667682390777 146.6856576 L 551.6816920597695 151.21730879999998 L 565.7650761793083 145.1870208 L 579.8484602988472 148.8871008 L 586.08 149.2546176" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> Virtual Node: 2-3 Swap Pure Distributed: 2-3 Swap 64 128 256 512 Number of Processes 0.00 0.01 0.02 0.03 0.04 <path d="M 621.6 155.5492032 L 625.4107912057298 158.79513599999996 L 636.1863621238082 157.9937472 L 650.7727242476162 157.1111424 L 671.8131810619041 156.670944 L 675.6239722676337 158.34499199999996 L 686.3995431857122 158.63439359999998 L 707.44 159.27943679999998 L 711.2507912057297 160.90018560000001 L 722.0263621238082 157.72662719999997 L 736.6127242476161 158.64698879999997 L 757.6531810619041 159.37470719999996 L 761.4639722676336 162.2544192 L 772.2395431857121 158.2073568 L 786.8259053095203 161.01135359999998 L 793.28 159.6499776 L 807.8663621238081 158.9270112 L 822.4527242476162 159.49195199999997 L 843.493181061904 159.40514879999998 L 858.0795431857122 159.58995839999997 L 872.6659053095202 160.20982080000002 L 879.12 158.9878944" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> Virtual Node: Remainder Pure Distributed: Remainder 64 128 256 512 Number of Processes 0.00 0.01 0.02 0.03 0.04

In [13]:
toyplot.pdf.render(canvas, 'vn-vs-smp-hdtv.pdf')

In [14]:
image_size = '8K UHD'

canvas = toyplot.Canvas('9.25in', '3in',
                        #style={'background-color': 'yellow'},
                       )

x_vn = average_partial_composite_vn.index
x_smp = average_partial_composite_smp.index

maxy = max(
    numpy.max(numpy.max(average_partial_composite_vn[image_size])),
    numpy.max(numpy.max(average_partial_composite_smp[image_size])),
)

xlocator = toyplot.locator.Log(base=2, format='{:.0f}')

def doPlot(axes, algorithm):
    axes.x.ticks.locator = xlocator
    axes.y.domain.min = 0
    axes.y.domain.max = maxy
    
    y_vn = numpy.array(average_partial_composite_vn[image_size, algorithm])
    y_smp = numpy.array(average_partial_composite_smp[image_size, algorithm])

    axes.plot(x_vn, y_vn)
    axes.plot(x_smp, y_smp)

    axes.text(
        512, numpy.max(y_vn), 'Virtual Node: ' + algorithm,
        style={
            'text-anchor': 'end',
            '-toyplot-vertical-align': 'last-baseline',
        }
    )
    axes.text(
        512, numpy.min(y_smp), 'Pure Distributed: ' + algorithm,
        style={
            'text-anchor': 'end',
            '-toyplot-vertical-align': 'top',
        }
    )

axesNaive = canvas.cartesian(
    xlabel = 'Number of Processes',
    ylabel = 'Partial Composite Time (seconds)',
    xscale='log',
    bounds=('5%', '33%', '5%', '85%'),
)
doPlot(axesNaive, 'Naive')

axes23 = canvas.cartesian(
    xlabel = 'Number of Processes',
    #ylabel = 'Partial Composite Time (seconds)',
    xscale='log',
    bounds=('38%', '66%', '5%', '85%'),
)
doPlot(axes23, '2-3 Swap')

axesRemainder = canvas.cartesian(
    xlabel = 'Number of Processes',
    #ylabel = 'Partial Composite Time (seconds)',
    xscale='log',
    bounds=('70%', '99%', '5%', '85%'),
)
doPlot(axesRemainder, 'Remainder')

<path d="M 44.400000000000006 155.6324352 L 48.07938461242868 80.41107455999999 L 58.4833841195389 72.88504319999998 L 72.56676823907773 72.43416575999997 L 92.88169205976945 72.02899967999998 L 96.56107667219807 73.94749439999997 L 106.96507617930834 73.38718207999997 L 127.28000000000002 160.24536064 L 130.9593846124287 81.16233216 L 141.3633841195389 75.65617151999996 L 155.44676823907767 75.37254912 L 175.76169205976947 76.72876031999998 L 179.441076672198 79.53580031999998 L 189.84507617930825 74.81209344000003 L 203.9284602988471 76.67119103999997 L 210.16000000000003 161.728768 L 224.24338411953892 78.07123968000002 L 238.3267682390777 78.12711936000004 L 258.64169205976947 78.73628160000003 L 272.72507617930825 79.09917695999997 L 286.80846029884714 78.24729599999996 L 293.04 161.65903871999998" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> Virtual Node: Naive Pure Distributed: Naive 64 128 256 512 Number of Processes 0.0 0.2 0.5 0.8 Partial Composite Time (seconds) <path d="M 337.44 150.23130624 L 341.1193846124287 149.17708799999997 L 351.5233841195389 146.22851072000006 L 365.60676823907767 148.1513472 L 385.92169205976944 151.73528575999998 L 389.601076672198 148.46647295999998 L 400.0050761793084 148.78200832 L 420.32000000000005 155.00860415999998 L 423.9993846124287 152.43331584 L 434.40338411953894 147.518208 L 448.4867682390777 150.05374464 L 468.8016920597695 153.40462080000003 L 472.481076672198 152.8134144 L 482.88507617930827 148.88847359999997 L 496.96846029884716 150.61304832000002 L 503.20000000000005 155.40867072 L 517.2833841195389 147.80066303999996 L 531.3667682390777 149.69210879999997 L 551.6816920597695 155.18518271999997 L 565.7650761793083 151.57787136 L 579.8484602988472 150.06587903999997 L 586.08 155.33458943999997" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> Virtual Node: 2-3 Swap Pure Distributed: 2-3 Swap 64 128 256 512 Number of Processes 0.0 0.2 0.5 0.8 <path d="M 621.6 156.03391487999997 L 625.4107912057298 157.52822784000003 L 636.1863621238082 157.91736831999995 L 650.7727242476162 156.35724288 L 671.8131810619041 158.19206656 L 675.6239722676337 157.0013184 L 686.3995431857122 159.30374144 L 707.44 160.29278208 L 711.2507912057297 161.13245184 L 722.0263621238082 159.63154944 L 736.6127242476161 159.2404224 L 757.6531810619041 160.04540416000003 L 761.4639722676336 161.69938943999998 L 772.2395431857121 159.02446079999999 L 786.8259053095203 160.63007231999998 L 793.28 161.17098496 L 807.8663621238081 160.70171136 L 822.4527242476162 161.25961728000001 L 843.493181061904 161.56010496 L 858.0795431857122 161.89244927999997 L 872.6659053095202 161.69438208 L 879.12 161.23754495999995" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> Virtual Node: Remainder Pure Distributed: Remainder 64 128 256 512 Number of Processes 0.0 0.2 0.5 0.8

In [15]:
toyplot.pdf.render(canvas, 'vn-vs-smp-8k.pdf')